In [6]:
from datasets import load_dataset

dataset = load_dataset("FinGPT/fingpt-sentiment-train")
dataset.save_to_disk("fingpt_sentiment_train")

# Optional: Save a specific split (e.g., 'train') to CSV
dataset['train'].to_csv("fingpt_sentiment_train.csv")

Saving the dataset (0/1 shards):   0%|          | 0/76772 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/77 [00:00<?, ?ba/s]

18311676

In [3]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 76772
    })
})


In [1]:
!pip install transformers==4.32.0 peft==0.5.0
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes


     ---------------------------------------- 0.0/118.5 kB ? eta -:--:--
     ---------- ---------------------------- 30.7/118.5 kB 1.4 MB/s eta 0:00:01
     ------------ ------------------------ 41.0/118.5 kB 495.5 kB/s eta 0:00:01
     ------------------------------------ 118.5/118.5 kB 993.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
    --------------------------------------- 0.1/7.5 MB 4.3 MB/s eta 0:00:02
   - -------------------------------------- 0.3/7.5 MB 3.2 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/7.5 MB 4.6 MB/s eta 0:00:02
   ---- ----------------------------------- 0.8/7.5 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.5 MB 5.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.5 MB 5.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.5 MB 5.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/7.5 MB 5.1 MB/s eta 0:00:02
   ------ 

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   - ------------------------------------- 30.7/991.5 kB 660.6 kB/s eta 0:00:02
   -- ------------------------------------ 71.7/991.5 kB 991.0 kB/s eta 0:00:01
   ---- ----------------------------------- 122.9/991.5 kB 1.2 MB/s eta 0:00:01
   ---- ----------------------------------- 122.9/991.5 kB 1.2 MB/s eta 0:00:01
   ----- -------------------------------- 143.4/991.5 kB 607.9 kB/s eta 0:00:02
   ------- ------------------------------ 204.8/991.5 kB 778.2 kB/s eta 0:00:02
   -------- ----------------------------- 225.3/991.5 kB 811.5 kB/s eta 0:00:01
   -------- ----------------------------- 225.3/991.5 kB 811.5 kB/s eta 0:00:01
   ---------- --------------------------- 286.7/991.5 kB 681.0 kB/s eta 0:00:02
   ------------ ------------------------- 337.9/991.5 kB 749.8 kB/s eta 0:00:01
   ----------------- -------------------- 450.6/991.5 kB 880.6 kB/s eta 0:00:01
   ---------------------- ----------------- 563.2

   ---------------------------------------- 0.0/136.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/136.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/136.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/136.5 MB 550.5 kB/s eta 0:04:08
   ---------------------------------------- 0.1/136.5 MB 804.6 kB/s eta 0:02:50
   ---------------------------------------- 0.2/136.5 MB 1.1 MB/s eta 0:02:09
   ---------------------------------------- 0.3/136.5 MB 1.4 MB/s eta 0:01:37
   ---------------------------------------- 0.5/136.5 MB 1.6 MB/s eta 0:01:26
   ---------------------------------------- 0.7/136.5 MB 2.1 MB/s eta 0:01:04
   ---------------------------------------- 0.9/136.5 MB 2.3 MB/s eta 0:01:01
   ---------------------------------------- 1.1/136.5 MB 2.6 MB/s eta 0:00:53
   ---------------------------------------- 1.4/136.5 MB 2.9 MB/s eta 0:00:47
   ---------------------------------------- 1.5/136.5 MB 3.0 MB/s eta 0:00:45
   

In [4]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0
# Load Models
# Load Models
base_model = "NousResearch/Llama-2-13b-hf" 
peft_model = "FinGPT/fingpt-sentiment_llama2-13b_lora"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True)
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()

# Make prompts
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .
Answer: ''',
]
# Generate results
tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512)
res = model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i, skip_special_tokens=True) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]

# Show results
for sentiment in out_text:
    print(sentiment)


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/196 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

C:\Users\PC\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
C:\Users\PC\anaconda3\Lib\site-packages\transformers\generation\utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


 positive
 neutral
 negative


In [1]:
@inproceedings{yang-etal-2019-read,
    title = "Read, Attend and Comment: A Deep Architecture for Automatic News Comment Generation",
    author = "Yang, Ze  and
      Xu, Can  and
      Wu, Wei  and
      Li, Zhoujun",
    booktitle = "Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP)",
    month = nov,
    year = "2019",
    address = "Hong Kong, China",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/D19-1512",
    doi = "10.18653/v1/D19-1512",
    pages = "5076--5088",
}

SyntaxError: unterminated string literal (detected at line 3) (1617567614.py, line 3)